# Accident Analysis in Metro Nashville: 2013-2017 (Part 1)

### This notebook will:
* read in the data for accidents in Metro Nashville for 2013-2017,
* clean up the data,
* extract features from the data,
* store the data in yearly files to be read in to Qlik for further analysis.


In [1]:
import glob
import os
import random
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 
import datetime
import time as tm
import numpy as np
from time import gmtime, strftime, localtime

pd.options.mode.chained_assignment = None


## Read in the data and fill in missing values

#### Read in data, concatenate into one big dataframe (omitting 2018 for now so that we only have full years), check column names, reset index and drop the old index column

In [2]:
%%time
path ='data'
allFiles = glob.glob(path + "/Metro_Nashville___Davidson_County_Traffic_Crashes*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    if file_ != "data/Metro_Nashville___Davidson_County_Traffic_Crashes__2018_.csv":
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
crash_data_df = pd.concat(list_)

CPU times: user 1.1 s, sys: 153 ms, total: 1.25 s
Wall time: 1.27 s


In [3]:
crash_data_df.columns

Index(['Accident Number', 'Date and Time', 'Number of Motor Vehicles',
       'Number of Injuries', 'Number of Fatalities', 'Property Damage',
       'Hit and Run', 'Reporting Officer', 'Collision Type',
       'Collision Type Description', 'Weather', 'Weather Description',
       'Illumination', 'Illumination Description', 'Harmful Codes',
       'Harmful Codes Description', 'Street Address', 'City', 'State', 'Zip',
       'RPA', 'Precinct', 'Latitude', 'Longitude', 'Mapped Location'],
      dtype='object')

In [4]:
crash_data_df = crash_data_df.reset_index().drop('index', axis=1)
#crash_data_df.fillna('missing', inplace=True)

#### Find nulls

In [5]:
crash_data_df.isnull().sum()

Accident Number                    0
Date and Time                      0
Number of Motor Vehicles          45
Number of Injuries                34
Number of Fatalities              48
Property Damage               103790
Hit and Run                       29
Reporting Officer                 48
Collision Type                    49
Collision Type Description       570
Weather                          243
Weather Description              243
Illumination                     533
Illumination Description         558
Harmful Codes                      0
Harmful Codes Description       1027
Street Address                   139
City                            1144
State                            742
Zip                             2700
RPA                             4120
Precinct                        4528
Latitude                        2835
Longitude                       2835
Mapped Location                    4
dtype: int64

#### If a record has neither street address nor lat/long, there's no way for me to deduce where it happened. How many are there?

In [6]:
crash_data_df[(crash_data_df['Street Address'].isnull() & crash_data_df['Latitude'].isnull())].shape

(84, 25)

#### Only 84 records -- not a big deal in the grand scheme of things. Dropping them.

In [7]:
crash_data_df = crash_data_df[~(crash_data_df['Street Address'].isnull() & crash_data_df['Latitude'].isnull())]

#### Assumptions for fixing nulls:
* Number of Injuries/Fatalities: if null, assume the answer is 0.
* Number of Motor Vehicles: if null, assume 1.
* Hit and Run: if null, assume N
* Weather: if null, assume 1 (no adverse conditions)
* Weather Description: if null, assume "NO ADVERSE CONDITIONS"
* Property Damage: if null, assume 'N'
* Illumination: if null... not sure. I'll have to look at time/location to see if I can determine the answer.
* Illumination Description: see above
* Street Address: if null, use Google Maps API and their lat/long to get the street address
* City: if null, use Google Maps API and their lat/long to get the city
* Precinct: if null, put 'UNKNOWN' for now and see if we can figure it out later
* Latitude/Longitude: if null, use Google Maps API and their street address to get the lat/long

In [8]:
crash_data_df.fillna({'Number of Injuries': 0,
                      'Number of Fatalities': 0,
                      'Hit and Run': 'N',
                      'Weather': 1,
                      'Weather Description': 'NO ADVERSE CONDITIONS',
                      'Precinct': 'UNKNOWN',
                      'Property Damage': 'N',
                      'Number of Motor Vehicles': 1,
                      'Reporting Officer': 0
                     }, inplace=True)

In [9]:
crash_data_df.isnull().sum()

Accident Number                  0
Date and Time                    0
Number of Motor Vehicles         0
Number of Injuries               0
Number of Fatalities             0
Property Damage                  0
Hit and Run                      0
Reporting Officer                0
Collision Type                  46
Collision Type Description     567
Weather                          0
Weather Description              0
Illumination                   530
Illumination Description       555
Harmful Codes                    0
Harmful Codes Description     1027
Street Address                  55
City                          1140
State                          738
Zip                           2694
RPA                           4039
Precinct                         0
Latitude                      2751
Longitude                     2751
Mapped Location                  0
dtype: int64

#### Change Date/Time to Timestamp

In [10]:
crash_data_df['Date and Time'] = pd.to_datetime(crash_data_df['Date and Time'])

#### FOR NOW, keep only datapoints for which we have reasonable lat/longs and street addresses. We'll come back and replace the bad ones later and pull this part out. 

In [11]:
non_empty_crash_data_df = crash_data_df[(crash_data_df['Latitude'] != 0) & 
                                        (~crash_data_df['Latitude'].isnull()) &
                                       (~crash_data_df['Street Address'].isnull())]
non_empty_crash_data_df

Accident Number       Date and Time  Number of Motor Vehicles  \
0          20130000050 2013-01-01 00:15:00                       2.0   
1          20130000270 2013-01-01 00:30:00                       2.0   
2          20130000128 2013-01-01 00:43:00                       2.0   
3          20130000123 2013-01-01 00:45:00                       2.0   
4          20130000160 2013-01-01 00:45:00                       2.0   
5          20130000142 2013-01-01 01:00:00                       2.0   
6          20130000138 2013-01-01 01:05:00                       1.0   
7          20130000206 2013-01-01 01:25:00                       2.0   
8          20130000306 2013-01-01 01:25:00                       2.0   
9          20130000393 2013-01-01 01:26:00                       1.0   
11         20130000249 2013-01-01 01:40:00                       2.0   
12         20130000240 2013-01-01 01:46:00                       2.0   
13         20130000255 2013-01-01 01:57:00                       1.0   
15         20130000315 2013-01-01 02:31:00                       1.0   
17         20130000379 2013-01-01 02:48:00                       2.0   
18         20130000391 2013-01-01 03:23:00                       1.0   
19         20130000403 2013-01-01 03:24:00                       2.0   
20         20130000408 2013-01-01 03:30:00                       2.0   
21         20130000414 2013-01-01 03:31:00                       2.0   
22         20130000407 2013-01-01 03:33:00                       1.0   
24         20130000532 2013-01-01 05:18:00                       1.0   
26         20130000564 2013-01-01 05:44:00                       1.0   
27         20130000637 2013-01-01 06:00:00                       1.0   
28         20130000662 2013-01-01 06:00:00                       1.0   
29         20130000626 2013-01-01 06:30:00                       1.0   
30         20130000627 2013-01-01 06:35:00                       1.0   
31         20130000635 2013-01-01 06:58:00                       1.0   
32         20130000651 2013-01-01 07:20:00                       1.0   
33         20130000671 2013-01-01 07:30:00                       2.0   
34         20130003167 2013-01-01 08:00:00                       2.0   
...                ...                 ...                       ...   
155243     20171139178 2017-12-31 14:06:00                       2.0   
155244     20171139249 2017-12-31 14:52:00                       2.0   
155245     20171139257 2017-12-31 14:44:00                       2.0   
155246     20171139265 2017-12-31 14:59:00                       2.0   
155247     20171139374 2017-12-31 14:43:00                       2.0   
155248     20171139391 2017-12-31 16:04:00                       2.0   
155249     20171139424 2017-12-31 16:20:00                       2.0   
155250     20171139461 2017-12-31 16:30:00                       2.0   
155251     20171139474 2017-12-31 16:25:00                       2.0   
155252     20171139500 2017-12-31 17:00:00                       2.0   
155253     20171139543 2017-12-31 17:15:00                       2.0   
155254     20171139573 2017-12-31 17:40:00                       2.0   
155255     20171139578 2017-12-31 00:00:00                       2.0   
155256     20171139596 2017-12-31 18:04:00                       2.0   
155257     20171139628 2017-12-31 18:29:00                       2.0   
155258     20171139672 2017-12-31 18:49:00                       2.0   
155259     20171139674 2017-12-31 00:00:00                       2.0   
155260     20171139680 2017-12-31 18:31:00                       2.0   
155261     20171139683 2017-12-31 19:02:00                       2.0   
155262     20171139706 2017-12-31 19:10:00                       2.0   
155263     20171139794 2017-12-31 20:14:00                       1.0   
155264     20171139809 2017-12-31 20:21:00                       1.0   
155265     20171139810 2017-12-31 20:06:00                       2.0   
155266     20171140033 2017-12-31 22:11:00     

## Extract Features

In [12]:
def round_time_and_loc(df):
    df['Rounded Date and Time'] = [time.replace(microsecond=0,second=0,minute=0) for time in df['Date and Time']]
    #df['Rounded Latitude'] = [round(lat, 3) for lat in df['Latitude']]
    #df['Rounded Longitude'] = [round(long, 3) for long in df['Longitude']]
    df['Location'] = list(zip(df['Latitude'], df['Longitude']))
    #df['Rounded Location'] = list(zip(df['Rounded Latitude'], df['Rounded Longitude']))

In [13]:
# time features extraction function
def extract_time_features(df):

    df['Day Of Week'] = [timestamp.dayofweek for timestamp in df['Rounded Date and Time']]
    df['Day Of Month'] = [timestamp.day for timestamp in df['Rounded Date and Time']]
    df['Month'] = [timestamp.month for timestamp in df['Rounded Date and Time']]
    df['Hour'] = [timestamp.hour for timestamp in df['Rounded Date and Time']]
    df['Year'] = [timestamp.year for timestamp in df['Rounded Date and Time']]
    df['Week'] = [timestamp.isocalendar()[1] for timestamp in df['Rounded Date and Time']]
    df['Weekend'] = [1 if timestamp.dayofweek in [5,6] else 0 for timestamp in df['Rounded Date and Time']]
    
    temp = []

    for hour, weekend_code in list(zip(df['Hour'], df['Weekend'])):
        if (hour in range(6,10)) & (weekend_code == 0):
            temp.append(1)
        else:
            temp.append(0)

    df['Rush Hour Morning'] = temp

    temp = []

    for hour, weekend_code in list(zip(df['Hour'], df['Weekend'])):
        if (hour in range(15,19)) & (weekend_code == 0):
            temp.append(1)
        else:
            temp.append(0)

    df['Rush Hour Afternoon'] = temp

In [14]:
# location features extraction function
def extract_address_features(df):
    df['Ramp'] = [1 if ('RAMP' in str(x)) or 
                  ('EXIT' in str(x)) or
                  ('EXT ' in str(x)) 
                  else 0 for x in df['Street Address']]
    df['Intersection'] = [1 if '&' in str(x) else 0 for x in df['Street Address']]
    df['Interstate'] = [1 if ('I24' in str(x)) or 
                           ('I 24' in str(x)) or 
                           ('I-24' in str(x)) or 
                           ('I40' in str(x)) or 
                           ('I 40' in str(x)) or 
                           ('I-40' in str(x)) or 
                           ('I65' in str(x)) or 
                           ('I 65' in str(x)) or 
                           ('I-65' in str(x)) or
                           ('I440' in str(x)) or
                        ('I 440' in str(x)) or
                        ('I-440' in str(x))
                           else 0 for x in df['Street Address']]


In [15]:
round_time_and_loc(non_empty_crash_data_df)
extract_time_features(non_empty_crash_data_df)
extract_address_features(non_empty_crash_data_df)

In [16]:
non_empty_crash_data_df.isnull().sum()

Accident Number                  0
Date and Time                    0
Number of Motor Vehicles         0
Number of Injuries               0
Number of Fatalities             0
Property Damage                  0
Hit and Run                      0
Reporting Officer                0
Collision Type                  45
Collision Type Description     511
Weather                          0
Weather Description              0
Illumination                   515
Illumination Description       538
Harmful Codes                    0
Harmful Codes Description     1017
Street Address                   0
City                           566
State                          612
Zip                            236
RPA                            660
Precinct                         0
Latitude                         0
Longitude                        0
Mapped Location                  0
Rounded Date and Time            0
Rounded Latitude                 0
Rounded Longitude                0
Location            

In [17]:
for x in [2013, 2014, 2015, 2016, 2017, 2018]:
    temp_df = non_empty_crash_data_df[non_empty_crash_data_df['Year'] == x]
    temp_df.to_csv('data/Qlik_Crash_Data_' + str(x) + '.csv')

In [18]:
non_empty_crash_data_df = non_empty_crash_data_df.dropna()

In [19]:
non_empty_crash_data_df.to_csv('data/Clean_Crash_Data.csv')

In [20]:
non_empty_crash_data_df

Accident Number       Date and Time  Number of Motor Vehicles  \
0          20130000050 2013-01-01 00:15:00                       2.0   
1          20130000270 2013-01-01 00:30:00                       2.0   
2          20130000128 2013-01-01 00:43:00                       2.0   
3          20130000123 2013-01-01 00:45:00                       2.0   
4          20130000160 2013-01-01 00:45:00                       2.0   
5          20130000142 2013-01-01 01:00:00                       2.0   
6          20130000138 2013-01-01 01:05:00                       1.0   
7          20130000206 2013-01-01 01:25:00                       2.0   
8          20130000306 2013-01-01 01:25:00                       2.0   
9          20130000393 2013-01-01 01:26:00                       1.0   
11         20130000249 2013-01-01 01:40:00                       2.0   
12         20130000240 2013-01-01 01:46:00                       2.0   
13         20130000255 2013-01-01 01:57:00                       1.0   
15         20130000315 2013-01-01 02:31:00                       1.0   
17         20130000379 2013-01-01 02:48:00                       2.0   
18         20130000391 2013-01-01 03:23:00                       1.0   
19         20130000403 2013-01-01 03:24:00                       2.0   
20         20130000408 2013-01-01 03:30:00                       2.0   
21         20130000414 2013-01-01 03:31:00                       2.0   
22         20130000407 2013-01-01 03:33:00                       1.0   
24         20130000532 2013-01-01 05:18:00                       1.0   
26         20130000564 2013-01-01 05:44:00                       1.0   
27         20130000637 2013-01-01 06:00:00                       1.0   
28         20130000662 2013-01-01 06:00:00                       1.0   
29         20130000626 2013-01-01 06:30:00                       1.0   
30         20130000627 2013-01-01 06:35:00                       1.0   
31         20130000635 2013-01-01 06:58:00                       1.0   
32         20130000651 2013-01-01 07:20:00                       1.0   
33         20130000671 2013-01-01 07:30:00                       2.0   
34         20130003167 2013-01-01 08:00:00                       2.0   
...                ...                 ...                       ...   
155243     20171139178 2017-12-31 14:06:00                       2.0   
155244     20171139249 2017-12-31 14:52:00                       2.0   
155245     20171139257 2017-12-31 14:44:00                       2.0   
155246     20171139265 2017-12-31 14:59:00                       2.0   
155247     20171139374 2017-12-31 14:43:00                       2.0   
155248     20171139391 2017-12-31 16:04:00                       2.0   
155249     20171139424 2017-12-31 16:20:00                       2.0   
155250     20171139461 2017-12-31 16:30:00                       2.0   
155251     20171139474 2017-12-31 16:25:00                       2.0   
155252     20171139500 2017-12-31 17:00:00                       2.0   
155253     20171139543 2017-12-31 17:15:00                       2.0   
155254     20171139573 2017-12-31 17:40:00                       2.0   
155255     20171139578 2017-12-31 00:00:00                       2.0   
155256     20171139596 2017-12-31 18:04:00                       2.0   
155257     20171139628 2017-12-31 18:29:00                       2.0   
155258     20171139672 2017-12-31 18:49:00                       2.0   
155259     20171139674 2017-12-31 00:00:00                       2.0   
155260     20171139680 2017-12-31 18:31:00                       2.0   
155261     20171139683 2017-12-31 19:02:00                       2.0   
155262     20171139706 2017-12-31 19:10:00                       2.0   
155263     20171139794 2017-12-31 20:14:00                       1.0   
155264     20171139809 2017-12-31 20:21:00                       1.0   
155265     20171139810 2017-12-31 20:06:00                       2.0   
155266     20171140033 2017-12-31 22:11:00     

In [21]:
non_empty_crash_data_df.columns

Index(['Accident Number', 'Date and Time', 'Number of Motor Vehicles',
       'Number of Injuries', 'Number of Fatalities', 'Property Damage',
       'Hit and Run', 'Reporting Officer', 'Collision Type',
       'Collision Type Description', 'Weather', 'Weather Description',
       'Illumination', 'Illumination Description', 'Harmful Codes',
       'Harmful Codes Description', 'Street Address', 'City', 'State', 'Zip',
       'RPA', 'Precinct', 'Latitude', 'Longitude', 'Mapped Location',
       'Rounded Date and Time', 'Rounded Latitude', 'Rounded Longitude',
       'Location', 'Rounded Location', 'Day Of Week', 'Day Of Month', 'Month',
       'Hour', 'Year', 'Week', 'Weekend', 'Rush Hour Morning',
       'Rush Hour Afternoon', 'Ramp', 'Intersection', 'Interstate'],
      dtype='object')

In [22]:
non_empty_crash_data_df.dtypes

Accident Number                       object
Date and Time                 datetime64[ns]
Number of Motor Vehicles             float64
Number of Injuries                   float64
Number of Fatalities                 float64
Property Damage                       object
Hit and Run                           object
Reporting Officer                     object
Collision Type                       float64
Collision Type Description            object
Weather                              float64
Weather Description                   object
Illumination                         float64
Illumination Description              object
Harmful Codes                         object
Harmful Codes Description             object
Street Address                        object
City                                  object
State                                 object
Zip                                  float64
RPA                                  float64
Precinct                              object
Latitude  

In [33]:
throwaway_df = non_empty_crash_data_df.loc[:,['Rounded Date and Time',
                                                                    'Location',
                                                                    'Street Address',
                                                                    'Weather Description',
                                                                    'Precinct',
                                                                    'Zip',
                                                                    'City']]

In [34]:
extract_time_features(throwaway_df)
extract_address_features(throwaway_df)

In [35]:
throwaway_df

Rounded Date and Time                                  Location  \
0        2013-01-01 00:00:00   (36.175273062828005, -86.7926580298143)   
1        2013-01-01 00:00:00                      (36.04522, -86.6591)   
2        2013-01-01 00:00:00     (36.0447764026146, -86.6615277299611)   
3        2013-01-01 00:00:00    (36.2179868590007, -86.84240625543941)   
4        2013-01-01 00:00:00                     (36.12465, -86.69971)   
5        2013-01-01 01:00:00    (36.1559017029874, -86.78301295923691)   
6        2013-01-01 01:00:00    (36.1579519522797, -86.78430846433528)   
7        2013-01-01 01:00:00                       (36.195, -86.70599)   
8        2013-01-01 01:00:00    (36.1537926678066, -86.79441039786231)   
9        2013-01-01 01:00:00     (36.2302139748994, -86.7899822394164)   
11       2013-01-01 01:00:00    (36.1821672516201, -86.81198092696859)   
12       2013-01-01 01:00:00     (36.0447764026146, -86.6615277299611)   
13       2013-01-01 01:00:00                     (36.16317, -86.78018)   
15       2013-01-01 02:00:00   (36.157081573867295, -86.7861695062386)   
17       2013-01-01 02:00:00     (36.2066708247587, -86.7784355026634)   
18       2013-01-01 03:00:00    (36.0821708416289, -86.65195258628859)   
19       2013-01-01 03:00:00   (36.075938079070504, -86.6599979129852)   
20       2013-01-01 03:00:00     (36.0457389397032, -86.6743502915592)   
21       2013-01-01 03:00:00     (36.1764247025516, -86.6720371074402)   
22       2013-01-01 03:00:00      (36.109514341666, -86.7217193330109)   
24       2013-01-01 05:00:00      (36.1908963993696, -86.774828366164)   
26       2013-01-01 05:00:00  (36.242796568798205, -86.78045880713002)   
27       2013-01-01 06:00:00   (36.042855133655706, -86.6065558201754)   
28       2013-01-01 06:00:00  (36.239911733864105, -86.87231406353321)   
29       2013-01-01 06:00:00    (36.07169970933629, -86.7163249348523)   
30       2013-01-01 06:00:00     (36.2919423982363, -86.8837742755729)   
31       2013-01-01 06:00:00    (36.1805131335695, -86.64499954229359)   
32       2013-01-01 07:00:00   (36.038996990503705, -86.7825804208461)   
33       2013-01-01 07:00:00     (36.1501556669256, -86.8579529201005)   
34       2013-01-01 08:00:00     (36.2801274801504, -86.7430199316633)   
...                      ...                                       ...   
155243   2017-12-31 14:00:00                       (36.1546, -86.7862)   
155244   2017-12-31 14:00:00           (36.197959999999995, -86.68785)   
155245   2017-12-31 14:00:00                     (36.16486, -86.77629)   
155246   2017-12-31 14:00:00                     (36.12757, -86.84388)   
155247   2017-12-31 14:00:00    (36.1701156411481, -86.77280158113442)   
155248   2017-12-31 16:00:00            (36.15775, -86.77318000000001)   
155249   2017-12-31 16:00:00           (36.169270000000004, -86.65555)   
155250   2017-12-31 16:00:00           (36.137859999999996, -86.82064)   
155251   2017-12-31 16:00:00                     (36.13057, -86.85638)   
155252   2017-12-31 17:00:00                     (36.07391, -86.92627)   
155253   2017-12-31 17:00:00            (36.06649, -86.69691999999999)   
155254   2017-12-31 17:00:00            (36.03739, -86.58639000000001)   
155255   2017-12-31 00:00:00            (36.16002, -86.82526999999999)   
155256   2017-12-31 18:00:00                     (36.24457, -86.73568)   
155257   2017-12-31 18:00:00                     (36.10742, -86.71966)   
155258   2017-12-31 18:00:00            (36.16817, -86.76590999999999)   
155259   2017-12-31 00:00:00            (36.16817, -86.76590999999999)   
155260   2017-12-31 18:00:00                     (36.04377, -86.95886)   
155261   2017-12-31 19:00:00  (36.154709999999994, -86.77311999999999)   
155262   2017-12-31 19:00:00            (36.22542, -86.86381999999999)   
155263   2017-12-31 20:00:00      (36.023722350999996, -86.6643558288)   
155264   2017-12-31 20:00:00            (36.09061, -86.94001999999999)   
155265   20

In [39]:
throwaway_df.dtypes

Rounded Date and Time    datetime64[ns]
Location                         object
Street Address                   object
Weather Description              object
Precinct                         object
Zip                             float64
City                             object
Day Of Week                       int64
Day Of Month                      int64
Month                             int64
Hour                              int64
Year                              int64
Week                              int64
Weekend                           int64
Rush Hour Morning                 int64
Rush Hour Afternoon               int64
Ramp                              int64
Intersection                      int64
Interstate                        int64
dtype: object

In [43]:
throwaway_df = throwaway_df.loc[:,['Weather Description', 'Precinct', 'Zip', 'City', 'Day Of Week', 'Day Of Month',
            'Month', 'Hour', 'Week', 'Weekend', 'Rush Hour Morning', 'Rush Hour Afternoon', 'Ramp',
            'Intersection', 'Interstate']].astype('category')

In [44]:
throwaway_df.dtypes

Weather Description    category
Precinct               category
Zip                    category
City                   category
Day Of Week            category
Day Of Month           category
Month                  category
Hour                   category
Week                   category
Weekend                category
Rush Hour Morning      category
Rush Hour Afternoon    category
Ramp                   category
Intersection           category
Interstate             category
dtype: object

In [47]:
throwaway_df = pd.get_dummies(throwaway_df)

In [48]:
throwaway_df

Weather Description_BLOWING SAND, SOIL, DIRT, SNOW  \
0                                                       0    
1                                                       0    
2                                                       0    
3                                                       0    
4                                                       0    
5                                                       0    
6                                                       0    
7                                                       0    
8                                                       0    
9                                                       0    
11                                                      0    
12                                                      0    
13                                                      0    
15                                                      0    
17                                                      0    
18                                                      0    
19                                                      0    
20                                                      0    
21                                                      0    
22                                                      0    
24                                                      0    
26                                                      0    
27                                                      0    
28                                                      0    
29                                                      0    
30                                                      0    
31                                                      0    
32                                                      0    
33                                                      0    
34                                                      0    
...                                                   ...    
155243                                                  0    
155244                                                  0    
155245                                                  0    
155246                                                  0    
155247                                                  0    
155248                                                  0    
155249                                                  0    
155250                                                  0    
155251                                                  0    
155252                                                  0    
155253                                                  0    
155254                                                  0    
155255                                                  0    
155256                                                  0    
155257                                                  0    
155258                                                  0    
155259                                                  0    
155260                                                  0    
155261                                                  0    
155262                                                  0    
155263                                                  0    
155264                                                  0    
155265                                                  0    
155266                                                  0    
155267                                                  0    
155268                                                  0    
155269                                                  0    
155270                                                  0    
155271                                                  0    
155272                                                  0    

        Weather Description_BLOWING SAND/SOIL/DIRT  \
0                                                0   
1                                                0   
2